In [1]:
## Loading the MHA and MQA models

import torch
import time
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer,AutoModelForCausalLM

C:\Users\DHIA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
## Models
MHA_MODEL = "gpt2-medium"
MQA_MODEL = "tiiuae/falcon-rw-1b"

## Tokenizers
tokenizer_mha = AutoTokenizer.from_pretrained(MHA_MODEL)
tokenizer_mqa = AutoTokenizer.from_pretrained(MQA_MODEL)

C:\Users\DHIA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DHIA\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\DHIA\AppData\Local\Packages\Py

In [7]:
prompt = "Your journey starts with one"

def get_attn_output(model_name,tokenizer,prompt):
    print(f"\nLOADING MODEL : {model_name}")
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                                                 low_cpu_mem_usage=True,
                                                 output_attention=True
                                                 )
    model.to(device)
    model.eval()
    inputs = tokenizer(prompt,return_tensors="pt").to(device)
    with torch.no_grad():
        start_time = time.time()
        outputs = model(inputs)
        end_time = time.time()
    latency = end_time - start_time
    return outputs.attentions,model,inputs,latency